In [1]:
import pandas as pd
import pyarc
from pyids import IDS
from pyids.algorithms.ids_classifier import mine_CARs
from pyids.rule_mining import RuleMiner
from pyarc.qcba.data_structures import QuantitativeDataFrame

from sklearn.model_selection import train_test_split

In [4]:
data = pd.read_csv("https://raw.githubusercontent.com/jirifilip/pyIDS/master/data/titanic.csv").sample(frac=1).reset_index(drop=True)

data_train, data_test = train_test_split(data, test_size=0.8)

In [5]:
rm = RuleMiner()
rules = rm.mine_rules(data_train)

[CAR {Age_Cat=adult} => {Died=0} sup: 0.95 conf: 0.00 len: 2, id: 930816, CAR {Age_Cat=adult} => {Died=1} sup: 0.95 conf: 0.00 len: 2, id: 930817, CAR {Gender=male} => {Died=0} sup: 0.80 conf: 0.00 len: 2, id: 930812, CAR {Gender=male} => {Died=1} sup: 0.80 conf: 0.00 len: 2, id: 930813, CAR {Gender=male,Age_Cat=adult} => {Died=0} sup: 0.77 conf: 0.00 len: 3, id: 930814, CAR {Gender=male,Age_Cat=adult} => {Died=1} sup: 0.77 conf: 0.00 len: 3, id: 930815, CAR {Passenger_Cat=crew} => {Died=0} sup: 0.39 conf: 0.00 len: 2, id: 930806, CAR {Passenger_Cat=crew} => {Died=1} sup: 0.39 conf: 0.00 len: 2, id: 930807, CAR {Passenger_Cat=crew,Age_Cat=adult} => {Died=0} sup: 0.39 conf: 0.00 len: 3, id: 930804, CAR {Passenger_Cat=crew,Age_Cat=adult} => {Died=1} sup: 0.39 conf: 0.00 len: 3, id: 930805, CAR {Passenger_Cat=crew,Gender=male} => {Died=0} sup: 0.38 conf: 0.00 len: 3, id: 930810, CAR {Passenger_Cat=crew,Gender=male} => {Died=1} sup: 0.38 conf: 0.00 len: 3, id: 930811, CAR {Gender=male,Pass

In [7]:
quant_dataframe_train = QuantitativeDataFrame(data_train)
quant_dataframe_test = QuantitativeDataFrame(data_test)

In [8]:
ids = IDS()
ids.fit(class_association_rules=rules, quant_dataframe=quant_dataframe_train, random_seed=5)

/Users/x/miniconda3/envs/orange3/lib/python3.10/site-packages/pyids-0.0.1-py3.10.egg/pyids/algorithms/ids_classifier.py:58: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


In [9]:
ids.clf.default_class

'0'

In [ ]:
print("Accuracy on train data: ", ids.score(quant_dataframe_train))
print("Accuracy on test data: ", ids.score(quant_dataframe_test))
print("AUC on train data: ", ids.score_auc(quant_dataframe_train))
print("AUC on test data: ", ids.score_auc(quant_dataframe_test))

In [ ]:
ids.score_interpretability_metrics(quant_dataframe_train)
ids.score_interpretability_metrics(quant_dataframe_test)

In [11]:
from pyids.model_selection import CoordinateAscent
from pyids.algorithms.ids import IDS
from pyids.algorithms import mine_CARs, mine_IDS_ruleset

from pyarc.qcba.data_structures import QuantitativeDataFrame

import pandas as pd
import numpy as np

df_iris = pd.read_csv("https://raw.githubusercontent.com/jirifilip/pyIDS/master/data/titanic.csv")
quant_df = QuantitativeDataFrame(df_iris)
cars = mine_CARs(df_iris, 20)


interpretability_bounds = dict(
    fraction_overlap=0.1,
    fraction_classes=1,
    fraction_uncovered=0.15,
    average_rule_width=8,
    ruleset_length=10
)

def is_solution_interpretable(metrics):
    print(metrics)
    return (
        metrics["fraction_overlap"] <= interpretability_bounds["fraction_overlap"] and
        metrics["fraction_classes"] >= interpretability_bounds["fraction_classes"] and
        metrics["fraction_uncovered"] <= interpretability_bounds["fraction_uncovered"] and
        metrics["average_rule_width"] <= interpretability_bounds["average_rule_width"] and
        metrics["ruleset_length"] <= interpretability_bounds["ruleset_length"]
    )

def solution_interpretability_distance(metrics):
    distance_vector = np.array([
        max(metrics["fraction_overlap"] - interpretability_bounds["fraction_overlap"], 0),
        max(interpretability_bounds["fraction_classes"] - metrics["fraction_classes"], 0),
        max(metrics["fraction_uncovered"] - interpretability_bounds["fraction_uncovered"], 0),
        max(metrics["average_rule_width"] - interpretability_bounds["average_rule_width"], 0),
        max(metrics["ruleset_length"] - interpretability_bounds["ruleset_length"], 0)
    ])
    return np.sum(distance_vector)
    #return np.linalg.norm(distance_vector)

def fmax(lambda_dict):
    print(lambda_dict)
    ids = IDS(algorithm="SLS")
    ids.fit(class_association_rules=cars, quant_dataframe=quant_df, lambda_array=list(lambda_dict.values()))

    metrics = ids.score_interpretability_metrics(quant_df)

    """
    if not is_solution_interpretable(metrics):
        distance = -solution_interpretability_distance(metrics)
        print(distance)
        return -distance
    """

    if not is_solution_interpretable(metrics):
        return 0

    auc = ids.score_auc(quant_df)

    print(auc)

    return auc



coord_asc = CoordinateAscent(
    func=fmax,
    func_args_ranges=dict(
        l1=(1, 1000),
        l2=(1, 1000),
        l3=(1, 1000),
        l4=(1, 1000),
        l5=(1, 1000),
        l6=(1, 1000),
        l7=(1, 1000)
    ),
    ternary_search_precision=50,
    max_iterations=2
)

best_pars = coord_asc.fit()

df = pd.DataFrame(coord_asc.procedure_data)

{'l1': 153, 'l2': 145, 'l3': 726, 'l4': 350, 'l5': 271, 'l6': 213, 'l7': 624}


/Users/x/miniconda3/envs/orange3/lib/python3.10/site-packages/pyids-0.0.1-py3.10.egg/pyids/algorithms/ids_classifier.py:58: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


{'fraction_overlap': 0.00017035775127768315, 'fraction_classes': 1.0, 'fraction_uncovered': 0.16922203293583193, 'average_rule_width': 2.2, 'ruleset_length': 5}
using precision 50
{'l1': 334.0, 'l2': 145, 'l3': 726, 'l4': 350, 'l5': 271, 'l6': 213, 'l7': 624}


/Users/x/miniconda3/envs/orange3/lib/python3.10/site-packages/pyids-0.0.1-py3.10.egg/pyids/algorithms/ids_classifier.py:58: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


{'fraction_overlap': 0.02701387198831832, 'fraction_classes': 1.0, 'fraction_uncovered': 0.1726291879613856, 'average_rule_width': 2.25, 'ruleset_length': 8}
{'l1': 667.0, 'l2': 145, 'l3': 726, 'l4': 350, 'l5': 271, 'l6': 213, 'l7': 624}


...

/Users/x/miniconda3/envs/orange3/lib/python3.10/site-packages/pyids-0.0.1-py3.10.egg/pyids/algorithms/ids_classifier.py:58: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/Users/x/miniconda3/envs/orange3/lib/python3.10/site-packages/pyids-0.0.1-py3.10.egg/pyids/algorithms/ids_classifier.py:58: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/Users/x/miniconda3/envs/orange3/lib/python3.10/site-packages/pyids-0.0.1-py3.10.egg/pyids/algorithms/ids_classifier.py:91: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


{'fraction_overlap': 0.09269624942538059, 'fraction_classes': 1.0, 'fraction_uncovered': 0.14253265190232822, 'average_rule_width': 2.0, 'ruleset_length': 7}
0.705186555092119
{'l1': 267.156378600823, 'l2': 471.6460905349794, 'l3': 119.15637860082305, 'l4': 387.44444444444446, 'l5': 271, 'l6': 213, 'l7': 624}


/Users/x/miniconda3/envs/orange3/lib/python3.10/site-packages/pyids-0.0.1-py3.10.egg/pyids/algorithms/ids_classifier.py:58: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/Users/x/miniconda3/envs/orange3/lib/python3.10/site-packages/pyids-0.0.1-py3.10.egg/pyids/algorithms/ids_classifier.py:58: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/Users/x/miniconda3/envs/orange3/lib/python3.10/site-packages/pyids-0.0.1-py3.10.egg/pyids/algorithms/ids_classifier.py:91: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


{'fraction_overlap': 0.06387063627268055, 'fraction_classes': 1.0, 'fraction_uncovered': 0.09596819988642813, 'average_rule_width': 2.142857142857143, 'ruleset_length': 7}
0.7095556838021622
{'l1': 267.156378600823, 'l2': 471.6460905349794, 'l3': 119.15637860082305, 'l4': 453.22222222222223, 'l5': 271, 'l6': 213, 'l7': 624}


KeyboardInterrupt: 

In [14]:
# Rounding the best parameters
pars = [round(x, 2) for x in best_pars]

# Creating the parameters dataframe
pars_df = pd.DataFrame({
    "Parameter": [f"$\\lambda_{i}$" for i in range(1, 8)],
    "Parameter Value": pars
})

# Printing the parameters dataframe as a LaTeX table
print(pars_df.to_latex(index=False, escape=False))

NameError: name 'best_pars' is not defined

In [ ]:
ids = IDS()
ids.fit(class_association_rules=rules, quant_dataframe=quant_dataframe_train, lambda_array=best_pars)
ids.score_auc(quant_dataframe_test)

In [13]:
metric_df = pd.DataFrame([
    {"Data": "Train", "Accuracy": ids.score(quant_dataframe_train), "AUC": ids.score_auc(quant_dataframe_train)},
    {"Data": "Test", "Accuracy": ids.score(quant_dataframe_test), "AUC": ids.score_auc(quant_dataframe_test)}
])

/Users/x/miniconda3/envs/orange3/lib/python3.10/site-packages/pyids-0.0.1-py3.10.egg/pyids/algorithms/ids_classifier.py:58: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/Users/x/miniconda3/envs/orange3/lib/python3.10/site-packages/pyids-0.0.1-py3.10.egg/pyids/algorithms/ids_classifier.py:58: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/Users/x/miniconda3/envs/orange3/lib/python3.10/site-packages/pyids-0.0.1-py3.10.egg/pyids/algorithms/ids_classifier.py:91: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/Users/x/miniconda3/envs/orange3/lib/python3.10/site-packages/pyids-0.0.1-py3.10.egg/pyids/algorithms/ids_classifier.py:58: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/Users/x/miniconda3/envs/orange3/lib/python3.10/site-packages/pyids-0.0.1-py3.10.egg/pyids/algorithms/id

In [ ]:
metric_df["Správnost"] = metric_df["Správnost"].apply(lambda x: round(x, 2))
metric_df["AUC"] = metric_df["AUC"].apply(lambda x: round(x, 2))
print(metric_df[["Data", "Správnost", "AUC"]].to_latex(index=False))

In [ ]:
ids.score_interpretability_metrics(quant_dataframe_test)

In [ ]:
procedure_data = coord_asc.procedure_data
procedure_df = pd.DataFrame(procedure_data).reset_index()
procedure_df